# [Bonus Tasks](svm_segm.ipynb)

In [13]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.ndimage as ndi
import skimage.morphology as morph
import skimage.util as sk
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [3]:
def blend_result(img, seg, border_radius=1, interior_opacity=1, interior_opacity_decay=0.9, color=(0,1,0)):
    img  = np.dstack([img] * 3).copy()
    img -= img.min()
    img /= img.max()
    selem  = morph.disk(border_radius)
    seg_bd = np.logical_xor(morph.binary_dilation(seg, selem), morph.binary_erosion(seg, selem))
    mask_decay = ndi.distance_transform_edt(seg)
    for i in range(3):
        opacity = interior_opacity / pow(1 + mask_decay[seg], interior_opacity_decay)
        img[:,:,i][seg] = color[i] * opacity + (1 - opacity) * img[:,:,i][seg]
        img[:,:,i][seg_bd] = color[i]
    return img

In [4]:
patch_sizes = (32, 32)

---

**[Task 1.1.]()** Implement `create_data_matrix`:

In [14]:
def create_data_matrix(img):
    img_datamatrix = sk.view_as_blocks(img, block_shape = (32, 32))
    return(img_datamatrix)

In [38]:
img = plt.imread('data/cells.png')
data_matrix = create_data_matrix(img)
data_matrix.shape
data_matrix[0][0][1]

(data_matrix[0][0] == data_matrix[0][1]).sum()



90

**[Task 1.2.]()** Implement `create_gt_labels_vector`:

In [41]:
def create_gt_labels_vector(ground_truth, patched_image):
    img_datamatrix = create_data_matrix(patched_image)
    truth_datamatrix = create_data_matrix(ground_truth)
    for i in range(data_matrix.shape[0]):
        for j in range(data_matrix.shape[1]):
            coverage = (img_data_matrix[i][j] == truth_datamatrix[i][j]).sum()
            relative_coverage = coverage//32*32
            if relative_coverage > 0.5: 
                img_datamatrix == 1
            else:
                img_datamatrix == -1
    return(img_datamatrix)



In [42]:
create_gt_labels_vector()

TypeError: create_gt_labels_vector() missing 2 required positional arguments: 'ground_truth' and 'patched_image'

**[Task 1.3.]()** Create the SVM classifier:

In [43]:
clf = make_pipeline(StandardScaler(), SVC(class_weight='balanced', gamma=0.1))

**[Task 1.3 (a).]()** Create the data matrices for the images `dna-33` and `dna-44`:

In [52]:
dna_33 = plt.imread("data/NIH3T3/im/dna-33.png")
dna_44 = plt.imread("data/NIH3T3/im/dna-44.png")
dna_33_gt = plt.imread("data/NIH3T3/gt/33.png")
dna_44_gt = plt.imread("data/NIH3T3/gt/44.png")

**[Task 1.3 (b).]()** Create the corresponding ground truth label vectors:

**[Task 1.3 (c).]()** Create the *combined* data matrices and ground truth label vectors:

**[Task 1.3 (d).]()** Train the classifier using the data matrix and label vectors from above:

**[Task 1.4.]()** Implement the function `predict_image`:

Test your implementation:

In [ ]:
img = plt.imread(f'data/NIH3T3/im/dna-0.png')
seg = predict_image(img)

plt.figure()
plt.imshow(blend_result(img, seg))

**[Task 1.5.]()** Perform batch processing: